In [1]:
import sys
sys.path.append("/home/dux/")
sys.path.append("/home/dux/surface_sampling/mcmc")

from ase.build import make_supercell
from ase.io import read, write

import catkit
from catkit.gen.adsorption import get_adsorption_sites

from mcmc import mcmc_run
import os
import numpy as np

os.environ["PROJECT_DIR"] = os.getcwd()

In [2]:
# SrTiO3 from HTVS db
import sys
import os
import django
import numpy as np
# from tqdm import tqdm
# import pandas as pd
# import pickle as pkl
sys.path.append('/home/dux/htvs/')
sys.path.append('/home/dux/htvs/djangochem/')

#from imports import *
#from django_imports import *
# from pymatgen.core.periodic_table import Element
# from chemconfigs.vasp.defaults import Magmom
# from ase.visualize import view

# setup the django settings file.  Change this to use the settings file that connects you to your desired database
os.environ["DJANGO_SETTINGS_MODULE"] = "djangochem.settings.orgel"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"  # CAUTION!!! LINE NECESSARY WHEN INTERACTING WITH DATABASE IN JUPYTER NOTEBOOK. DO NOT USE THIS LINE IF YOU ARE PLANNING ON SAVING/REMOVING THINGS FROM THE DATABASE!!!!!

# this must be run to setup access to the django settings and make database access work etc.
django.setup()
# from jobs.models import Job, JobConfig
# from django.contrib.contenttypes.models import ContentType
# from ase.io import write

from pgmols.models import Surface
# from analysis.structure.perovskite import PerovskiteAnalyzer
# from pymatgen.core.periodic_table import Element

unit_slab = Surface.objects.get(id=217485271).as_ase_gratoms()
unit_slab.write('SrTiO3_001_pristine_slab.cif')

# get supercell
# multiplier = np.diag([2, 2, 1])
# slab = make_supercell(unit_slab, multiplier)
slab = unit_slab*(2,2,1)
slab.write('SrTiO3_001_2x2_pristine_slab.cif')

In [3]:
# Pymatgen adsorption sites for InGaN
from pymatgen.analysis.adsorption import AdsorbateSiteFinder
from pymatgen.io.ase import AseAtomsAdaptor

from ase.io import read, write

pristine_slab = slab
pristine_pmg_slab = AseAtomsAdaptor.get_structure(pristine_slab)
site_finder = AdsorbateSiteFinder(pristine_pmg_slab)
# AdsorbateSiteFinder(slab, selective_dynamics=False, height=0.9, mi_vec=None)
sites = site_finder.find_adsorption_sites(put_inside=False, symm_reduce=False)
# find_adsorption_sites(distance=2.0, put_inside=True, symm_reduce=0.01, near_reduce=0.01, positions=['ontop', 'bridge', 'hollow'], no_obtuse_hollow=True)

In [4]:
for type in sites.keys():
    print(f"{type} sites has total {len(sites[type])}")

ontop sites has total 12
bridge sites has total 36
hollow sites has total 16
all sites has total 64


In [5]:
ads_positions = sites['all']
# print(f"adsorption coordinates are")
# print(ads_positions)
# take just the first 18 sites after inspection
# select_positions = ads_positions[:18]

# take all positions
select_positions = ads_positions

In [6]:
# adsorb onto the sites
from pymatgen.core.structure import Molecule
ads_mol = Molecule(['O'], [[0,0,0]])
for ads_coord in select_positions:
    slab = site_finder.add_adsorbate(ads_mol, ads_coord, repeat=None, translate=False, reorient=False)
    site_finder.slab = slab

from pymatgen.io.cif import CifWriter
# write out slab
cif_slab = CifWriter(site_finder.slab)
# cif_slab.write_file("pymatgen_SrTiO3_001_4x4_O_all_adsorbed_slab.cif")
cif_slab.write_file("pymatgen_SrTiO3_001_2x2_O_all_adsorbed_slab.cif")

In [16]:
all_adsorbed = read("pymatgen_SrTiO3_001_2x2_O_all_adsorbed_slab.cif")
print(f"all adsorbed {len(all_adsorbed)}")

select_positions = all_adsorbed.get_positions()[len(slab):]

all adsorbed 124


In [18]:
element = 'O'
ads = catkit.gratoms.Gratoms(element)

2x2 superslab

In [21]:
# do from 36 to 48 oxygen atoms
# so add a pristine slab
start_atoms = 36
end_atoms = 48
step = 1
scaling = (2,2,1)
num_sweeps = 20

# for i in range(start_atoms+step, end_atoms, step):
i = start_atoms+1 # try just 1 O adsorbate
print(f"going for {i} atoms")
num_ads_atoms =  i - start_atoms # for canonical runs
alpha = 1
slab = unit_slab*scaling
surface_name = f"pymatgen_SrTiO3_001_2x2"

ads_positions = select_positions

history, energy_hist, frac_accept_hist, adsorption_count_hist = mcmc_run(num_sweeps=num_sweeps, temp=1, pot=0, alpha=alpha, slab=slab, calc=None, testing=True, surface_name=surface_name, element=element, canonical=True, ads_coords=ads_positions, num_ads_atoms=num_ads_atoms)

going for 37 atoms


For 2x2 collate atoms

In [22]:
import glob
# collate items
# for 2x2
# structure_folders = glob.glob("pymatgen_SrTiO3_001_4x4/runs100_temp1_adsatoms??_alpha1*")
structure_folders = glob.glob("pymatgen_SrTiO3_001_2x2/runs20_temp1_adsatoms??_alpha1*")
structure_folders.sort()
print(structure_folders)
assert len(structure_folders) == 1

['pymatgen_SrTiO3_001_2x2/runs20_temp1_adsatoms01_alpha1_20220628-190920']


In [23]:
folder = structure_folders[0]
ads_slabs = glob.glob(folder+"/*")
ads_slabs.sort()
print(ads_slabs)

['pymatgen_SrTiO3_001_2x2/runs20_temp1_adsatoms01_alpha1_20220628-190920/final_slab_run_001.cif', 'pymatgen_SrTiO3_001_2x2/runs20_temp1_adsatoms01_alpha1_20220628-190920/final_slab_run_002.cif', 'pymatgen_SrTiO3_001_2x2/runs20_temp1_adsatoms01_alpha1_20220628-190920/final_slab_run_003.cif', 'pymatgen_SrTiO3_001_2x2/runs20_temp1_adsatoms01_alpha1_20220628-190920/final_slab_run_004.cif', 'pymatgen_SrTiO3_001_2x2/runs20_temp1_adsatoms01_alpha1_20220628-190920/final_slab_run_005.cif', 'pymatgen_SrTiO3_001_2x2/runs20_temp1_adsatoms01_alpha1_20220628-190920/final_slab_run_006.cif', 'pymatgen_SrTiO3_001_2x2/runs20_temp1_adsatoms01_alpha1_20220628-190920/final_slab_run_007.cif', 'pymatgen_SrTiO3_001_2x2/runs20_temp1_adsatoms01_alpha1_20220628-190920/final_slab_run_008.cif', 'pymatgen_SrTiO3_001_2x2/runs20_temp1_adsatoms01_alpha1_20220628-190920/final_slab_run_009.cif', 'pymatgen_SrTiO3_001_2x2/runs20_temp1_adsatoms01_alpha1_20220628-190920/final_slab_run_010.cif', 'pymatgen_SrTiO3_001_2x2/runs

In [24]:
# slabs = ["final_slab_run_010.cif", "final_slab_run_030.cif", "final_slab_run_050.cif", "final_slab_run_070.cif", "final_slab_run_090.cif"]
# ads_slabs = []
# for f in structure_folders:
#     for slab in slabs:
#         ads_slabs.append(os.path.join(f, slab))

import shutil
collate_path = "pymatgen_SrTiO3_001_2x2/20_random"
if not os.path.exists(collate_path):
    os.makedirs(collate_path)
for i, slab in enumerate(ads_slabs):
    move_path = os.path.join(collate_path, f"slab_{i+1:03}.cif")
    print(f"move from {slab} to {move_path}")
    shutil.copy2(slab, move_path)

move from pymatgen_SrTiO3_001_2x2/runs20_temp1_adsatoms01_alpha1_20220628-190920/final_slab_run_001.cif to pymatgen_SrTiO3_001_2x2/20_random/slab_001.cif
move from pymatgen_SrTiO3_001_2x2/runs20_temp1_adsatoms01_alpha1_20220628-190920/final_slab_run_002.cif to pymatgen_SrTiO3_001_2x2/20_random/slab_002.cif
move from pymatgen_SrTiO3_001_2x2/runs20_temp1_adsatoms01_alpha1_20220628-190920/final_slab_run_003.cif to pymatgen_SrTiO3_001_2x2/20_random/slab_003.cif
move from pymatgen_SrTiO3_001_2x2/runs20_temp1_adsatoms01_alpha1_20220628-190920/final_slab_run_004.cif to pymatgen_SrTiO3_001_2x2/20_random/slab_004.cif
move from pymatgen_SrTiO3_001_2x2/runs20_temp1_adsatoms01_alpha1_20220628-190920/final_slab_run_005.cif to pymatgen_SrTiO3_001_2x2/20_random/slab_005.cif
move from pymatgen_SrTiO3_001_2x2/runs20_temp1_adsatoms01_alpha1_20220628-190920/final_slab_run_006.cif to pymatgen_SrTiO3_001_2x2/20_random/slab_006.cif
move from pymatgen_SrTiO3_001_2x2/runs20_temp1_adsatoms01_alpha1_20220628-19

Check slabs to make sure they are distinct

In [25]:
from ase.utils.structure_comparator import SymmetryEquivalenceCheck
comp = SymmetryEquivalenceCheck()
import pickle
import glob

In [26]:
# load 2x2 structures
slab_paths = glob.glob("pymatgen_SrTiO3_001_2x2/20_random/*")
slab_paths.sort()
slabs = [read(x) for x in slab_paths]

# before tagging
slabs[1].write("pymatgen_SrTiO3_001_2x2_1ads_before_tagging.cif") # check for shifted cells

# set tags; 1 for surface atoms, 2 for adsorbates, 0 for others
superslab = unit_slab*(2,2,1)
surface_atoms = superslab.get_surface_atoms()
atoms_arr = np.arange(0, len(superslab))
base_tags = np.int8(np.isin(atoms_arr, surface_atoms)).tolist()
print(base_tags)
for slab in slabs:
    num_ads_atoms = len(slab) - len(superslab)
    tags = list(base_tags)
    tags.extend([2]*num_ads_atoms)
    slab.set_tags(tags)

# after tagging
slabs[1].write("pymatgen_SrTiO3_001_2x2_1ads_after_tagging.cif") # check for shifted cells

# run only once
equivalent_slab = []
for i in range(len(slabs)):
    for j in range(i+1, len(slabs)):
        if comp.compare(slabs[i], slabs[j]):
            equivalent_slab.append((i,j))

equivalent_slab
# so exclude 1 and 2

[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1]


[(0, 7),
 (0, 13),
 (1, 6),
 (1, 11),
 (1, 18),
 (3, 5),
 (3, 10),
 (3, 12),
 (3, 16),
 (5, 10),
 (5, 12),
 (5, 16),
 (6, 11),
 (6, 18),
 (7, 13),
 (8, 19),
 (9, 17),
 (10, 12),
 (10, 16),
 (11, 18),
 (12, 16),
 (14, 15)]

In [28]:
chosen_slabs = slabs.copy()
del chosen_slabs[1]
del chosen_slabs[1]
assert len(chosen_slabs) == len(slabs) - 2

print(f"slab length {len(chosen_slabs)}")
with open("pymatgen_SrTiO3_001_4x4_O_ads_slabs.pkl", "wb") as f:
    pickle.dump(chosen_slabs, f)

slab length 53


In [29]:
# check slabs are doing ok
# load pkl file
import pickle
with open("SrTiO3_001_2x2_O_ads_slabs.pkl", 'rb') as f:
    cell_2x2 = pickle.load(f)
    cell_2x2[0].write("SrTiO3_001_2x2_pris_from_pkl.cif")
    cell_2x2[1].write("SrTiO3_001_2x2_1ads_from_pkl.cif")
    cell_2x2[-1].write("SrTiO3_001_2x2_moreads_from_pkl.cif")

In [30]:
# load 3x3 structures
slab_paths = glob.glob("SrTiO3_001_3x3_filtered/collated/*")
slab_paths.sort()
slabs = [read(x) for x in slab_paths]
# set tags; 1 for surface atoms, 2 for adsorbates, 0 for others
superslab = unit_slab*(3,3,1)
surface_atoms = superslab.get_surface_atoms()
atoms_arr = np.arange(0, len(superslab))
base_tags = np.int8(np.isin(atoms_arr, surface_atoms)).tolist()
print(base_tags)
for slab in slabs:
    num_ads_atoms = len(slab) - len(superslab)
    tags = list(base_tags)
    tags.extend([2]*num_ads_atoms)
    slab.set_tags(tags)

# run only once
# equivalent_slab = []
# for i in range(len(slabs)):
#     for j in range(i+1, len(slabs)):
#         if comp.compare(slabs[i], slabs[j]):
#             equivalent_slab.append((i,j))

# equivalent_slab
# so exclude 1

[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1]


In [31]:
chosen_slabs = slabs.copy()
del chosen_slabs[1]
assert len(chosen_slabs) == len(slabs) - 1

print(f"slab length {len(chosen_slabs)}")
with open("SrTiO3_001_3x3_O_ads_slabs.pkl", "wb") as f:
    pickle.dump(chosen_slabs, f)

slab length 51


In [32]:
# check slabs are doing ok
# load pkl file
import pickle
with open("SrTiO3_001_3x3_O_ads_slabs.pkl", 'rb') as f:
    cell_3x3 = pickle.load(f)
    cell_3x3[0].write("SrTiO3_001_3x3_pris_from_pkl.cif")
    cell_3x3[1].write("SrTiO3_001_3x3_1ads_from_pkl.cif")
    cell_3x3[-1].write("SrTiO3_001_3x3_moreads_from_pkl.cif")

In [33]:
# load 4x4 structures
slab_paths = glob.glob("SrTiO3_001_4x4_filtered/collated/*")
slab_paths.sort()
slabs = [read(x) for x in slab_paths]
# set tags; 1 for surface atoms, 2 for adsorbates, 0 for others
superslab = unit_slab*(4,4,1)
surface_atoms = superslab.get_surface_atoms()
atoms_arr = np.arange(0, len(superslab))
base_tags = np.int8(np.isin(atoms_arr, surface_atoms)).tolist()
print(base_tags)
for slab in slabs:
    num_ads_atoms = len(slab) - len(superslab)
    tags = list(base_tags)
    tags.extend([2]*num_ads_atoms)
    slab.set_tags(tags)

print(f"slab length {len(slabs)}")
with open("SrTiO3_001_4x4_O_ads_slabs.pkl", "wb") as f:
    pickle.dump(slabs, f)

[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1]
slab length 23


In [34]:
# check slabs are doing ok
# load pkl file
import pickle
with open("SrTiO3_001_4x4_O_ads_slabs.pkl", 'rb') as f:
    cell_4x4 = pickle.load(f)
    cell_4x4[0].write("SrTiO3_001_4x4_pris_from_pkl.cif")
    cell_4x4[1].write("SrTiO3_001_4x4_1ads_from_pkl.cif")
    cell_4x4[-1].write("SrTiO3_001_4x4_moreads_from_pkl.cif")